In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 1.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425345 sha256=9a0ded4826902b5373ae8b683f14b7fc252d3364d72e903f781e010e952f5527
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [123]:
import pandas as pd
import random
import os
import sys
from pyspark.context import SparkContext
from pyspark.conf import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.types import *
import pyspark.sql.functions as F
import pyspark.sql.types as pst
from pyspark.ml.feature import Bucketizer
import copy
import pandas as pd
from sklearn.metrics import roc_auc_score
from tqdm import tqdm

In [124]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**促活场景，可以不发优惠券**

In [125]:
user_id = [3790260]*4 + [4687825]*4 + [5077519]*4 + [7927329]*4
coupon_id = [2060114740, 1806204031, 4096652575, 1234832524] * 4
coupon_reduce = [20, 10, 10, 0] * 4
coupon_threshold = [60, 60, 40, -1] * 4
is_convert = []
control_convert = []

In [126]:
for _ in range(4):
  for _ in range(3):
    is_convert.append(random.uniform(0.005, 0.3))
  temp = random.uniform(0, 0.005)
  is_convert.append(temp)
  control_convert.extend([temp]*4)

In [127]:
data = {'user_id': user_id, 'coupon_id': coupon_id, 'coupon_reduce': coupon_reduce, 'coupon_threshold': coupon_threshold, 'is_convert': is_convert, 'control_convert':control_convert}

In [128]:
df_old = pd.DataFrame(data)
df_old['expected_threshold_uplift'] = (df_old['is_convert'] - df_old['control_convert']) * df_old['coupon_threshold']

In [129]:
df_old

,user_id,coupon_id,coupon_reduce,coupon_threshold,is_convert,control_convert,expected_threshold_uplift
0,3790260,2060114740,20,60,0.233967,0.003082,13.853114
1,3790260,1806204031,10,60,0.173529,0.003082,10.226854
2,3790260,4096652575,10,40,0.163917,0.003082,6.433426
3,3790260,1234832524,0,-1,0.003082,0.003082,-0.000000
4,4687825,2060114740,20,60,0.275729,0.000279,16.527020
5,4687825,1806204031,10,60,0.133412,0.000279,7.988009
6,4687825,4096652575,10,40,0.275069,0.000279,10.991610
7,4687825,1234832524,0,-1,0.000279,0.000279,-0.000000
8,5077519,2060114740,20,60,0.187636,0.001924,11.142722
9,5077519,1806204031,10,60,0.183453,0.001924,10.891729


**拉新场景，都需要发优惠券**

In [130]:
user_id = [3790260]*4 + [4687825]*4 + [5077519]*4 + [7927329]*4
coupon_id = [2060114740, 1806204031, 4096652575, 1234832524] * 4
coupon_reduce = [20, 10, 10, 0] * 4
coupon_threshold = [98, 98, 68, -1] * 4
is_convert = []
control_convert = []

In [131]:
for _ in range(4):
  for _ in range(3):
    is_convert.append(random.uniform(0.005, 0.3))
  temp = 0
  is_convert.append(temp)
  control_convert.extend([temp]*4)

In [132]:
data = {'user_id': user_id, 'coupon_id': coupon_id, 'coupon_reduce': coupon_reduce, 'coupon_threshold': coupon_threshold, 'is_convert': is_convert, 'control_convert':control_convert}

In [133]:
df_new = pd.DataFrame(data)
df_new['expected_threshold_uplift'] = (df_new['is_convert'] - df_new['control_convert']) * df_new['coupon_threshold']

In [134]:
df_new

,user_id,coupon_id,coupon_reduce,coupon_threshold,is_convert,control_convert,expected_threshold_uplift
0,3790260,2060114740,20,98,0.171505,0,16.807529
1,3790260,1806204031,10,98,0.042032,0,4.119103
2,3790260,4096652575,10,68,0.195480,0,13.292630
3,3790260,1234832524,0,-1,0.000000,0,-0.000000
4,4687825,2060114740,20,98,0.130384,0,12.777594
5,4687825,1806204031,10,98,0.273902,0,26.842371
6,4687825,4096652575,10,68,0.032596,0,2.216498
7,4687825,1234832524,0,-1,0.000000,0,-0.000000
8,5077519,2060114740,20,98,0.044864,0,4.396632
9,5077519,1806204031,10,98,0.231278,0,22.665269


**转为pyspark计算**

In [135]:
# 初始化spark会话
spark = SparkSession.builder.getOrCreate()

In [136]:
spark_df_old = spark.createDataFrame(df_old)
spark_df_old.show()

+-------+----------+-------------+----------------+--------------------+--------------------+-------------------------+
|user_id| coupon_id|coupon_reduce|coupon_threshold|          is_convert|     control_convert|expected_threshold_uplift|
+-------+----------+-------------+----------------+--------------------+--------------------+-------------------------+
|3790260|2060114740|           20|              60| 0.23396696208735995|0.003081724383531...|       13.853114262229724|
|3790260|1806204031|           10|              60| 0.17352928847960694|0.003081724383531...|       10.226853845764543|
|3790260|4096652575|           10|              40| 0.16391737573102966|0.003081724383531...|        6.433426053899938|
|3790260|1234832524|            0|              -1|0.003081724383531...|0.003081724383531...|                     -0.0|
|4687825|2060114740|           20|              60|  0.2757289275003614|2.786015915007423E-4|       16.527019554531638|
|4687825|1806204031|           10|      

In [137]:
spark_df_new = spark.createDataFrame(df_new)
spark_df_new.show()

+-------+----------+-------------+----------------+--------------------+---------------+-------------------------+
|user_id| coupon_id|coupon_reduce|coupon_threshold|          is_convert|control_convert|expected_threshold_uplift|
+-------+----------+-------------+----------------+--------------------+---------------+-------------------------+
|3790260|2060114740|           20|              98|  0.1715053967759044|              0|       16.807528884038632|
|3790260|1806204031|           10|              98|0.042031663648089315|              0|        4.119103037512753|
|3790260|4096652575|           10|              68| 0.19547985868769374|              0|       13.292630390763174|
|3790260|1234832524|            0|              -1|                 0.0|              0|                     -0.0|
|4687825|2060114740|           20|              98| 0.13038360772451696|              0|       12.777593557002662|
|4687825|1806204031|           10|              98| 0.27390174015734736|        

In [138]:
result_df_old = spark_df_old.groupby('user_id').agg(
    F.collect_list('is_convert').alias('is_convert'),
    F.collect_list('coupon_id').alias('coupon_id'),
    F.collect_list('coupon_reduce').alias('coupon_reduce'),
    F.collect_list('coupon_threshold').alias('coupon_threshold'),
    F.collect_list('expected_threshold_uplift').alias('expected_threshold_uplift'),).cache()

result_df_old.show()

+-------+--------------------+--------------------+---------------+----------------+-------------------------+
|user_id|          is_convert|           coupon_id|  coupon_reduce|coupon_threshold|expected_threshold_uplift|
+-------+--------------------+--------------------+---------------+----------------+-------------------------+
|3790260|[0.23396696208735...|[2060114740, 1806...|[20, 10, 10, 0]|[60, 60, 40, -1]|     [13.8531142622297...|
|4687825|[0.27572892750036...|[2060114740, 1806...|[20, 10, 10, 0]|[60, 60, 40, -1]|     [16.5270195545316...|
|5077519|[0.18763644920778...|[2060114740, 1806...|[20, 10, 10, 0]|[60, 60, 40, -1]|     [11.1427218020453...|
|7927329|[0.09589937759791...|[2060114740, 1806...|[20, 10, 10, 0]|[60, 60, 40, -1]|     [5.74236140901221...|
+-------+--------------------+--------------------+---------------+----------------+-------------------------+



In [139]:
result_df_new = spark_df_new.groupby('user_id').agg(
    F.collect_list('is_convert').alias('is_convert'),
    F.collect_list('coupon_id').alias('coupon_id'),
    F.collect_list('coupon_reduce').alias('coupon_reduce'),
    F.collect_list('coupon_threshold').alias('coupon_threshold'),
    F.collect_list('expected_threshold_uplift').alias('expected_threshold_uplift'),).cache()

result_df_new.show()

+-------+--------------------+--------------------+---------------+----------------+-------------------------+
|user_id|          is_convert|           coupon_id|  coupon_reduce|coupon_threshold|expected_threshold_uplift|
+-------+--------------------+--------------------+---------------+----------------+-------------------------+
|3790260|[0.17150539677590...|[2060114740, 1806...|[20, 10, 10, 0]|[98, 98, 68, -1]|     [16.8075288840386...|
|4687825|[0.13038360772451...|[2060114740, 1806...|[20, 10, 10, 0]|[98, 98, 68, -1]|     [12.7775935570026...|
|5077519|[0.04486359551504...|[2060114740, 1806...|[20, 10, 10, 0]|[98, 98, 68, -1]|     [4.39663236047393...|
|7927329|[0.00695453419686...|[2060114740, 1806...|[20, 10, 10, 0]|[98, 98, 68, -1]|     [0.68154435129238...|
+-------+--------------------+--------------------+---------------+----------------+-------------------------+



**MCKP求解**

In [140]:
epoch = 1000
B = 20
C = 1 * B
alpha = 0.01
lr_decay = 0.98

In [141]:
def MCKP(df, epoch, B, C, alpha, lr_decay=1):
    N = df.count()
    lambda_B = 0
    lambda_C = 0

    for _ in tqdm(range(epoch)):

        temp = df.cache()

        # 固定lambda，更新xij，选择收益最大的优惠券，lamda_B * B为常量，没有计算进来
        temp = temp.withColumn('score', F.expr('zip_with(is_convert, coupon_reduce, (x, y) -> (x - %f * y))' % (lambda_B)))
        temp = temp.withColumn('score', F.expr('zip_with(score, expected_threshold_uplift, (x, y) -> (x + %f * y))' % (lambda_C)))
        temp = temp.withColumn('choice', F.expr('array_position(score, array_max(score)) - 1'))

        # 固定优惠券，更新lambda
        delta_B = temp.select(F.sum(F.col('coupon_reduce')[F.col('choice')])).collect()[0][0]
        lambda_B = max(0, lambda_B - alpha * (B - delta_B)/N)

        delta_C = temp.select(F.sum(F.col('expected_threshold_uplift')[F.col('choice')])).collect()[0][0]
        lambda_C = max(0, lambda_C - alpha * (delta_C - C)/N)

        total_budget = temp.select(F.sum(F.col('coupon_reduce')[F.col('choice')]).alias('total_budget')).toPandas()['total_budget'][0]
        budget_offset = total_budget / B - 1

        if _ % 100 == 0:
            print("epoch: {0}, lambda_B: {1}, lambda_C: {2}, budget_offset: {3}".format(_, lambda_B, lambda_C, budget_offset))

        alpha *= lr_decay
        temp.unpersist()

    print("lambda_B: {0}, lambda_C: {1}, budget_offset: {2}".format(lambda_B, lambda_C, budget_offset))
    result = temp.withColumn('choose_incentive', F.col('coupon_id')[F.col('choice')])
    result.groupby('choose_incentive').count().show()

    return temp

In [142]:
result_old = MCKP(result_df_old, epoch, B, C, alpha, lr_decay)

  0%|          | 1/1000 [00:00<04:17,  3.88it/s]

epoch: 0, lambda_B: 0.125, lambda_C: 0, budget_offset: 2.5


 10%|█         | 101/1000 [00:25<04:01,  3.72it/s]

epoch: 100, lambda_B: 0.031139924315292465, lambda_C: 0.012075579032960602, budget_offset: -0.5


 20%|██        | 201/1000 [00:52<03:42,  3.60it/s]

...

epoch: 800, lambda_B: 0.02936050579976468, lambda_C: 0.006057606023294031, budget_offset: 0.0


 90%|█████████ | 902/1000 [03:28<00:19,  5.15it/s]

epoch: 900, lambda_B: 0.02936050579976468, lambda_C: 0.0060576014966649844, budget_offset: 0.0


100%|██████████| 1000/1000 [03:51<00:00,  4.32it/s]


lambda_B: 0.02936050579976468, lambda_C: 0.006057600898218651, budget_offset: 0.0
+----------------+-----+
|choose_incentive|count|
+----------------+-----+
|      1234832524|    2|
|      4096652575|    2|
+----------------+-----+



In [143]:
result_old = result_old.withColumn('choose_coupon', F.col('coupon_id')[F.col('choice')]) \
                        .withColumn('choose_budget', F.col('coupon_reduce')[F.col('choice')]) \
                        .withColumn('choose_cost', F.col('coupon_threshold')[F.col('choice')])

In [144]:
result_old.show()

+-------+--------------------+--------------------+---------------+----------------+-------------------------+--------------------+------+-------------+-------------+-----------+
|user_id|          is_convert|           coupon_id|  coupon_reduce|coupon_threshold|expected_threshold_uplift|               score|choice|choose_coupon|choose_budget|choose_cost|
+-------+--------------------+--------------------+---------------+----------------+-------------------------+--------------------+------+-------------+-------------+-----------+
|3790260|[0.23396696208735...|[2060114740, 1806...|[20, 10, 10, 0]|[60, 60, 40, -1]|     [13.8531142622297...|[-0.2693308717120...|     3|   1234832524|            0|         -1|
|4687825|[0.27572892750036...|[2060114740, 1806...|[20, 10, 10, 0]|[60, 60, 40, -1]|     [16.5270195545316...|[-0.2113703880382...|     2|   4096652575|           10|         40|
|5077519|[0.18763644920778...|[2060114740, 1806...|[20, 10, 10, 0]|[60, 60, 40, -1]|     [11.142721802045

In [145]:
epoch = 1000
B = 30
C = 1 * B
alpha = 0.01
lr_decay = 0.98

In [146]:
result_new = MCKP(result_df_new, epoch, B, C, alpha, lr_decay)

  0%|          | 1/1000 [00:00<03:25,  4.87it/s]

epoch: 0, lambda_B: 0.025, lambda_C: 0, budget_offset: 0.33333333333333326


 10%|█         | 102/1000 [00:21<02:48,  5.32it/s]

epoch: 100, lambda_B: 0.025, lambda_C: 0, budget_offset: 0.0


 20%|██        | 201/1000 [00:43<03:22,  3.94it/s]

...

epoch: 800, lambda_B: 0.025, lambda_C: 0, budget_offset: 0.0


 90%|█████████ | 902/1000 [03:10<00:18,  5.30it/s]

epoch: 900, lambda_B: 0.025, lambda_C: 0, budget_offset: 0.0


100%|██████████| 1000/1000 [03:30<00:00,  4.74it/s]


lambda_B: 0.025, lambda_C: 0, budget_offset: 0.0
+----------------+-----+
|choose_incentive|count|
+----------------+-----+
|      1234832524|    1|
|      4096652575|    2|
|      1806204031|    1|
+----------------+-----+



In [147]:
result_new = result_new.withColumn('choose_coupon', F.col('coupon_id')[F.col('choice')]) \
                        .withColumn('choose_budget', F.col('coupon_reduce')[F.col('choice')]) \
                        .withColumn('choose_cost', F.col('coupon_threshold')[F.col('choice')])

In [148]:
result_new.show()

+-------+--------------------+--------------------+---------------+----------------+-------------------------+--------------------+------+-------------+-------------+-----------+
|user_id|          is_convert|           coupon_id|  coupon_reduce|coupon_threshold|expected_threshold_uplift|               score|choice|choose_coupon|choose_budget|choose_cost|
+-------+--------------------+--------------------+---------------+----------------+-------------------------+--------------------+------+-------------+-------------+-----------+
|3790260|[0.17150539677590...|[2060114740, 1806...|[20, 10, 10, 0]|[98, 98, 68, -1]|     [16.8075288840386...|[-0.3284946032240...|     3|   1234832524|            0|         -1|
|4687825|[0.13038360772451...|[2060114740, 1806...|[20, 10, 10, 0]|[98, 98, 68, -1]|     [12.7775935570026...|[-0.3696163922754...|     1|   1806204031|           10|         98|
|5077519|[0.04486359551504...|[2060114740, 1806...|[20, 10, 10, 0]|[98, 98, 68, -1]|     [4.3966323604739

In [149]:
def solve_binary_search(df, epoch, B, C):
    N = df.count()
    lambda_B = 0
    lambda_B_lb = 0
    lambda_B_ub = 10
    epsilon = 1e-6

    for _ in tqdm(range(epoch)):

        temp = df.cache()
        lambda_B = (lambda_B_lb + lambda_B_ub) / 2

        # 固定lambda，更新xij，选择收益最大的优惠券，lamda_B * B为常量，没有计算进来
        temp = temp.withColumn('score', F.expr('zip_with(is_convert, coupon_reduce, (x, y) -> (x - %f * y))' % (lambda_B)))
        temp = temp.withColumn('choice', F.expr('array_position(score, array_max(score)) - 1'))

        total_budget = temp.select(F.sum(F.col('coupon_reduce')[F.col('choice')]).alias('total_budget')).toPandas()['total_budget'][0]
        budget_offset = total_budget / B - 1

        print("epoch: {0}, lambda_B: {1}, lambda_B_lb: {2}, lambda_B_ub:{3}, budget_offset: {4}".format(_, lambda_B, lambda_B_lb, lambda_B_ub, budget_offset))

        if abs(budget_offset) < epsilon:
          break
        elif total_budget < B:
          lambda_B_ub = lambda_B
        else:
          lambda_B_lb = lambda_B


        temp.unpersist()

    print("lambda_B: {0}, budget_offset: {1}".format(lambda_B,budget_offset))

    print("choose_incentive: ")
    result = temp.withColumn('choose_incentive', F.col('coupon_id')[F.col('choice')])
    result.groupby('choose_incentive').count().show()

    return temp

In [150]:
epoch = 1000
B = 20
C = 1 * B

In [151]:
result_old = solve_binary_search(result_df_old, epoch, B, C)

  0%|          | 2/1000 [00:00<03:09,  5.27it/s]

epoch: 0, lambda_B: 5.0, lambda_B_lb: 0, lambda_B_ub:10, budget_offset: -1.0
epoch: 1, lambda_B: 2.5, lambda_B_lb: 0, lambda_B_ub:5.0, budget_offset: -1.0


  0%|          | 4/1000 [00:00<02:23,  6.95it/s]

epoch: 2, lambda_B: 1.25, lambda_B_lb: 0, lambda_B_ub:2.5, budget_offset: -1.0
epoch: 3, lambda_B: 0.625, lambda_B_lb: 0, lambda_B_ub:1.25, budget_offset: -1.0


  1%|          | 6/1000 [00:00<01:53,  8.77it/s]

epoch: 4, lambda_B: 0.3125, lambda_B_lb: 0, lambda_B_ub:0.625, budget_offset: -1.0
epoch: 5, lambda_B: 0.15625, lambda_B_lb: 0, lambda_B_ub:0.3125, budget_offset: -1.0
epoch: 6, lambda_B: 0.078125, lambda_B_lb: 0, lambda_B_ub:0.15625, budget_offset: -1.0


  1%|          | 8/1000 [00:01<02:18,  7.14it/s]


epoch: 7, lambda_B: 0.0390625, lambda_B_lb: 0, lambda_B_ub:0.078125, budget_offset: -1.0
epoch: 8, lambda_B: 0.01953125, lambda_B_lb: 0, lambda_B_ub:0.0390625, budget_offset: 0.0
lambda_B: 0.01953125, budget_offset: 0.0
choose_incentive: 
+----------------+-----+
|choose_incentive|count|
+----------------+-----+
|      1234832524|    2|
|      4096652575|    2|
+----------------+-----+



In [152]:
result_old = result_old.withColumn('choose_coupon', F.col('coupon_id')[F.col('choice')]) \
                        .withColumn('choose_budget', F.col('coupon_reduce')[F.col('choice')]) \
                        .withColumn('choose_cost', F.col('coupon_threshold')[F.col('choice')])

In [153]:
result_old.show()

+-------+--------------------+--------------------+---------------+----------------+-------------------------+--------------------+------+-------------+-------------+-----------+
|user_id|          is_convert|           coupon_id|  coupon_reduce|coupon_threshold|expected_threshold_uplift|               score|choice|choose_coupon|choose_budget|choose_cost|
+-------+--------------------+--------------------+---------------+----------------+-------------------------+--------------------+------+-------------+-------------+-----------+
|3790260|[0.23396696208735...|[2060114740, 1806...|[20, 10, 10, 0]|[60, 60, 40, -1]|     [13.8531142622297...|[-0.1566530379126...|     3|   1234832524|            0|         -1|
|4687825|[0.27572892750036...|[2060114740, 1806...|[20, 10, 10, 0]|[60, 60, 40, -1]|     [16.5270195545316...|[-0.1148910724996...|     2|   4096652575|           10|         40|
|5077519|[0.18763644920778...|[2060114740, 1806...|[20, 10, 10, 0]|[60, 60, 40, -1]|     [11.142721802045

In [154]:
epoch = 1000
B = 30
C = 1 * B

In [155]:
result_new = solve_binary_search(result_df_new, epoch, B, C)

  0%|          | 1/1000 [00:00<01:56,  8.56it/s]

epoch: 0, lambda_B: 5.0, lambda_B_lb: 0, lambda_B_ub:10, budget_offset: -1.0


  0%|          | 2/1000 [00:00<01:46,  9.33it/s]

epoch: 1, lambda_B: 2.5, lambda_B_lb: 0, lambda_B_ub:5.0, budget_offset: -1.0
epoch: 2, lambda_B: 1.25, lambda_B_lb: 0, lambda_B_ub:2.5, budget_offset: -1.0
epoch: 3, lambda_B: 0.625, lambda_B_lb: 0, lambda_B_ub:1.25, budget_offset: -1.0

  1%|          | 6/1000 [00:00<01:35, 10.40it/s]

...

epoch: 6, lambda_B: 0.078125, lambda_B_lb: 0, lambda_B_ub:0.15625, budget_offset: -1.0
epoch: 7, lambda_B: 0.0390625, lambda_B_lb: 0, lambda_B_ub:0.078125, budget_offset: -1.0
epoch: 8, lambda_B: 0.01953125, lambda_B_lb: 0, lambda_B_ub:0.0390625, budget_offset: 0.33333333333333326

  1%|          | 10/1000 [00:01<01:43,  9.59it/s]


epoch: 9, lambda_B: 0.029296875, lambda_B_lb: 0.01953125, lambda_B_ub:0.0390625, budget_offset: -1.0


  1%|          | 10/1000 [00:01<01:51,  8.87it/s]


epoch: 10, lambda_B: 0.0244140625, lambda_B_lb: 0.01953125, lambda_B_ub:0.029296875, budget_offset: 0.0
lambda_B: 0.0244140625, budget_offset: 0.0
choose_incentive: 
+----------------+-----+
|choose_incentive|count|
+----------------+-----+
|      1234832524|    1|
|      4096652575|    2|
|      1806204031|    1|
+----------------+-----+



In [156]:
result_new = result_new.withColumn('choose_coupon', F.col('coupon_id')[F.col('choice')]) \
                        .withColumn('choose_budget', F.col('coupon_reduce')[F.col('choice')]) \
                        .withColumn('choose_cost', F.col('coupon_threshold')[F.col('choice')])

In [157]:
result_new.show()

+-------+--------------------+--------------------+---------------+----------------+-------------------------+--------------------+------+-------------+-------------+-----------+
|user_id|          is_convert|           coupon_id|  coupon_reduce|coupon_threshold|expected_threshold_uplift|               score|choice|choose_coupon|choose_budget|choose_cost|
+-------+--------------------+--------------------+---------------+----------------+-------------------------+--------------------+------+-------------+-------------+-----------+
|3790260|[0.17150539677590...|[2060114740, 1806...|[20, 10, 10, 0]|[98, 98, 68, -1]|     [16.8075288840386...|[-0.3167746032240...|     3|   1234832524|            0|         -1|
|4687825|[0.13038360772451...|[2060114740, 1806...|[20, 10, 10, 0]|[98, 98, 68, -1]|     [12.7775935570026...|[-0.3578963922754...|     1|   1806204031|           10|         98|
|5077519|[0.04486359551504...|[2060114740, 1806...|[20, 10, 10, 0]|[98, 98, 68, -1]|     [4.3966323604739